# Capstone Project - The Battle of Neighborhoods (Week 1-2)

## Introduction

### Background

London is considered to be one of the world's most important global cities and has been called the world's most powerful, most desirable, most influential, most visited, most expensive, innovative, sustainable, most investment-friendly, and most-popular-for-work city.

London attract lots of new inhabitants, new investors each year because above reasons.

### Problem

London is a huge city, not everyone known London well before they come there. It is risky to move to a place you barely known.

This report is a investment guide to people who want to open a shop or restaurant in London. 

It supposed to help those people to have a general business view of London. So, they can better decide where they are going.

### Assumptions

Usually businesses are grouped together, people like to go to clustered shops or restaurant to have a wider range of choices hence more efficient shopping/eating experience. If you want to open a shop/restaurant, I assume it is safer to go to those places which already have shops/restaurants to minimize the risk.

Besides that, richer neighborhoods usually have higher spending power, you would like to setup your business there to maximize the revenue. In this case, I assume higher house sales price will indicate higher spending power.

So, this report are based on two assumptions:
1. More similar business at the same place indicates lesser risk.
2. Higher average house sales price indicates higher revenue.


## Data

To achieve the goal of minimal risk and maximal revenue. I need different sources of data.

- List of boroughs can be obtained on Wikipedia. https://en.wikipedia.org/wiki/List_of_London_boroughs
    - Borough names and geo coordinates can be found here, geo coordinates will be used in foursquare API to find recommended venues.
- Areas list on Wikipedia https://en.wikipedia.org/wiki/List_of_areas_of_London
    - Postcode to boroughs mapping can be extracted here, Postcode will be used in house sales data to find house price of that postcode then that borough.
- By using foursquare API, list of recommended venues can be fetched base on above coordinates. https://api.foursquare.com/v2/venues/explore
    - List of top venues of each borough can be found here to decide the functionality of a borough
- House sale price data on Land registry of UK, I will use most recent data from year 2019. http://prod1.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-2019.csv
    - Postcode to house sale price mapping can be extracted here.


In [1]:
#!wget http://prod1.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-2019.csv

--2020-03-15 21:22:08--  http://prod1.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-2019.csv
Resolving prod1.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com (prod1.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com)... 52.218.65.58
Connecting to prod1.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com (prod1.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com)|52.218.65.58|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162089122 (155M) [text/csv]
Saving to: ‘pp-2019.csv’

100%[======================================>] 162,089,122 72.4MB/s   in 2.1s   

2020-03-15 21:22:10 (72.4 MB/s) - ‘pp-2019.csv’ saved [162089122/162089122]



In [2]:
import pandas as pd

In [17]:
df = pd.read_csv('pp-2019.csv', names=['TUID', 'Price', 'Date_Transfer', 'Postcode', 'Prop_Type', 'Old_New', 'Duration', 'PAON', 'SAON', 'Street', 'Locality', 'Town_City', 'District', 'County', 'PPD_Cat_Type', 'Record_Status'])
df.head()

,TUID,Price,Date_Transfer,Postcode,Prop_Type,Old_New,Duration,PAON,SAON,Street,Locality,Town_City,District,County,PPD_Cat_Type,Record_Status
0,{8355F009-BD69-55C5-E053-6B04A8C0D090},130000,2019-01-14 00:00,BH2 5PS,O,N,L,44 - 46,NaN,POOLE HILL,NaN,BOURNEMOUTH,BOURNEMOUTH,BOURNEMOUTH,B,A
1,{8355F009-BD6C-55C5-E053-6B04A8C0D090},396000,2019-01-31 00:00,SP8 4HB,O,N,F,DOLPHIN INN,NaN,PEACEMARSH,NaN,GILLINGHAM,NORTH DORSET,DORSET,B,A
2,{8355F009-BD6D-55C5-E053-6B04A8C0D090},197500,2019-01-11 00:00,SP8 4EQ,D,N,F,14,NaN,SYLVAN WAY,NaN,GILLINGHAM,NORTH DORSET,DORSET,B,A
3,{8355F009-BD6E-55C5-E053-6B04A8C0D090},230000,2019-01-28 00:00,BH4 9JY,F,N,L,"WHEATON GRANGE, 16",FLAT 6,BRANKSOME WOOD ROAD,NaN,BOURNEMOUTH,BOURNEMOUTH,BOURNEMOUTH,B,A
4,{8355F009-BD6F-55C5-E053-6B04A8C0D090},215500,2019-01-17 00:00,BH12 1AW,F,N,L,344,FLAT 1,POOLE ROAD,BRANKSOME,POOLE,POOLE,POOLE,B,A


In [16]:
df_london = df[df['city'] == 'LONDON']
df_london.shape

(57316, 16)